In [1]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
import numpy as np
import json
import os

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

Create the estimator

In [2]:
tf_estimator = TensorFlow(entry_point='train.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m4.xlarge',
                          framework_version='2.4.1', 
                          sagemaker_session=sagemaker_session,
                          py_version='py37',
                          script_mode=True,
                          hyperparameters={ 'epochs': 2 },
                         )

Start the training

In [3]:
tf_estimator.fit({'train': 's3://bird-sound-data/data/'})

2021-09-01 07:45:55 Starting - Starting the training job...
2021-09-01 07:45:59 Starting - Launching requested ML instancesProfilerReport-1630482354: InProgress
...
2021-09-01 07:46:54 Starting - Preparing the instances for training............
2021-09-01 07:48:40 Downloading - Downloading input data...
2021-09-01 07:49:17 Training - Downloading the training image..2021-09-01 07:49:31.037656: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-09-01 07:49:31.045201: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-09-01 07:49:31.263873: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-09-01 07:49:37,608 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-09-01 07:49:37,615 sagemaker-traini


2021-09-01 07:50:18 Uploading - Uploading generated training model
2021-09-01 07:50:18 Completed - Training job completed
Training seconds: 89
Billable seconds: 89


Deploy the model

In [4]:
predictor = tf_estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---------------!

Deploy the model from the artifacts

In [2]:
from sagemaker.tensorflow import TensorFlowModel
model_uri = 's3://sagemaker-us-east-1-495733045912/tensorflow-training-2021-09-01-07-45-54-294/output/model.tar.gz'
model = TensorFlowModel(model_data=model_uri, framework_version='2.4.1',  role=role)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------!

Setting the autoscaling

In [8]:
client = boto3.client('application-autoscaling')

In [9]:
resource_id='endpoint/' + predictor.endpoint_name + '/variant/' + 'AllTraffic' # This is the format in which application autoscaling references the endpoint

In [14]:
response = client.register_scalable_target(
    ServiceNamespace='sagemaker', #
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    MinCapacity=1,
    MaxCapacity=4
)

In [15]:
response = client.put_scaling_policy(
    PolicyName='CPUUtil-ScalingPolicy',
    ServiceNamespace='sagemaker',
    ResourceId=resource_id,
    ScalableDimension='sagemaker:variant:DesiredInstanceCount',
    PolicyType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration={
        'TargetValue': 300,
        'CustomizedMetricSpecification':
        {
            'MetricName': 'CPUUtilization',
            'Namespace': '/aws/sagemaker/Endpoints',
            'Dimensions': [
                {'Name': 'EndpointName', 'Value': predictor.endpoint_name },
                {'Name': 'VariantName','Value': 'AllTraffic'}
            ],
            'Statistic': 'Average',
            'Unit': 'Percent'
        },
        'ScaleInCooldown': 300,
        'ScaleOutCooldown': 300
    }
)